## 1. Getting VizDoom up and running

In [6]:
# import VizDoom for game env
from vizdoom import *
# Import random for action sampling
import random
# Import time for sleeping
import time
# import numpy for identity matrix
import numpy as np

from matplotlib import pyplot as plt

## 2. Converting it to a Gym Environment

In [7]:
# Import environment base class from OpenAI Gym
from gymnasium import Env
# Import gym spaces
from gymnasium.spaces import Discrete, Box
# Import Opencv for greyscaling observations
import cv2

# Import environment checker
from stable_baselines3.common import env_checker
# Discrete(3).sample() returns a number from 0, 1, 2 -> used as index to select action
# Box(low=0, high=10, shape=(10,10)).sample() -> getting 10x10 array with low=0 and high=10

In [8]:
# Create VizDoom OpenAI Gym Environment
class VizDoomGym(Env): 
    def __init__(self, render=False):
        """
        Function called when we start the env.
        """

        # Inherit from Env
        super().__init__()
        
        # Set up game
        self.game = DoomGame()
        self.game.load_config('VizDoom/scenarios/basic.cfg')
        

        # Whether we want to render the game 
        if render == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)

        # Start the game
        self.game.init()
        
        # Create action space and observation space
        self.observation_space = Box(low=0, high=255, shape=(100, 160, 1), dtype=np.uint8)
        self.action_space = Discrete(3)

    
    def step(self, action):
        """
        How we take a step in the environment.
        """

        # Specify action and take step
        actions = np.identity(3, dtype=np.uint8)
        reward = self.game.make_action(actions[action], 4) # get action using index -> left, right, shoot
        
        # Get all the other stuff we need to return 
        if self.game.get_state():  # if nothing is
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)  # Apply Grayscale
            ammo = self.game.get_state().game_variables[0] 
            info = ammo
        # If we dont have anything turned from game.get_state
        else:
            # Return a numpy zero array
            state = np.zeros(self.observation_space.shape)
            # Return info (game variables) as zero
            info = 0

        info = {"info":info}
        done = self.game.is_episode_finished()
        truncated = False  # Assuming it's not truncated, modify if applicable
        
        return state, reward, done, truncated, info

    
    def render(self):
        """
        Define how to render the game environment.
        """
        pass

    
    def reset(self, seed=None):
        """
        Function for defining what happens when we start a new game.
        """
        if seed is not None:
            self.game.set_seed(seed)
            
        self.game.new_episode()
        state = self.game.get_state().screen_buffer  # Apply Grayscale

        return self.grayscale(state), {}

    
    def grayscale(self, observation):
        """
        Function to grayscale the game frame and resize it.
        observation: gameframe
        """
        # Change colour channels 
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)

        # Reduce image pixel size for faster training
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize,(100, 160,1))
        return state

    def close(self):
        """
        Call to close down the game.
        """
        self.game.close()


In [4]:
env = VizDoomGym(render=True)

Environment checker

In [5]:
env_checker.check_env(env)

In [6]:
env.close()

## 3. Setup Callback
Save model at different state of training

In [9]:
# Import os for file nav
import os
# Import callback class from sb3
from stable_baselines3.common.callbacks import BaseCallback

In [10]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        return True
                                      

In [11]:
CHECKPOINT_DIR = './train/train_basic'
LOG_DIR = './logs/log_basic'

In [12]:
# Create instance of callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR) 
# after every 10000 steps of training the model, weights are saved for the pytorch agent

## 5. Train Model


In [13]:
# Import PPO for training
from stable_baselines3 import PPO

In [14]:
# Non rendered environment
env = VizDoomGym()

In [15]:
# n_steps: How many steps/frames the agent is going to take and store in the buffer 
# before run through training of actor and critique
# ideally not too close to end of game (300) but somewhere close
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.0001, n_steps=2048)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [16]:
model.learn(total_timesteps=100000, callback=callback)

Logging to ./logs/log_basic/PPO_3


KeyboardInterrupt: 

## 5. Test Model


In [ ]:
# Import eval policy to test agent
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
# Reload model from disc
model = PPO.load('./train/train_basic/best_model_110000')

In [ ]:
# Create rendered envrironment
env = VizDoomGym(render=True)

In [ ]:
# Evaluate mean reward for 10 games
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=10)

In [ ]:
mean_reward

85.4

In [ ]:
total_reward = 0
for episode in range(5):
    obs = env.reset()[0]
    done = False
    while not done:
        action, _ = model.predict(obs) # Use model to predict what action to take
        obs, reward, done, _, info = env.step(action) # take the predicted action
        time.sleep(0.25)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(total_reward, episode))
    time.sleep(2)
          
    

Total Reward for episode 91.0 is 0
Total Reward for episode 170.0 is 1
Total Reward for episode 249.0 is 2
Total Reward for episode 340.0 is 3
Total Reward for episode 407.0 is 4
